In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler


import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./train.csv')
info = pd.read_csv('./data_info.csv')

In [3]:
train.head()

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,1,106,지방도1112호선,0,0,60.0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,2,103,일반국도11호선,0,0,60.0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,2,103,일반국도16호선,0,0,80.0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,2,107,태평로,0,0,50.0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


In [4]:
dictionary = {}
for i in range(24):
    key = info.iloc[i, 0]
    value = info.iloc[i, 1]
    
    dictionary[key] = value
    
train = train.rename(columns = dictionary)

In [5]:
train.columns

Index(['아이디', '날짜', '요일', '시간대', '차로수', '도로등급', '도로명', '중용구간 여부', '연결로 코드',
       '최고속도제한', '통과제한차량', '통과제한하중', '통과제한높이', '도로유형', '시작지점명', '시작지점의 위도',
       '시작지점의 경도', '시작 지점의 회전제한 유무', '도착지점명', '도착지점의 위도', '도착지점의 경도',
       '도작지점의 회전제한 유무', '평균속도(km)'],
      dtype='object')

In [6]:
a = train[['최고속도제한', '도로등급', '통과제한하중', '연결로 코드',
           '차로수', '시작지점의 위도', '시작지점의 경도', '시작지점명', '시작 지점의 회전제한 유무']]


columns_to_scale = ['최고속도제한', '연결로 코드', '통과제한하중', '도로등급']

# MinMaxScaler 적용
scaler = MinMaxScaler()
a_scaled = scaler.fit_transform(a[columns_to_scale])

# 스케일링된 값을 a 데이터프레임에 할당
a[columns_to_scale] = a_scaled


In [7]:
a['최고속도제한'] *= 0.3437
a['도로등급'] *= 0.1827
a['통과제한하중'] *= 0.087
a['연결로 코드'] *= 0.0618

In [8]:
a['열의 합'] = a['최고속도제한'] + a['도로등급'] + a['통과제한하중'] + a['연결로 코드']

In [9]:
a.drop(['최고속도제한', '연결로 코드', '통과제한하중', '도로등급'], 
           axis=1, inplace=True)

In [11]:
a.drop(['시작지점명', '시작 지점의 회전제한 유무'], 
           axis=1, inplace=True)

In [13]:
a.drop(['차로수'], 
           axis=1, inplace=True)

In [16]:
a.head()

,시작지점의 위도,시작지점의 경도,열의 합
0,33.427747,126.662612,0.399621
1,33.500730,126.529107,0.206220
2,33.279145,126.368598,0.343700
3,33.246081,126.567204,0.320180
4,33.462214,126.326551,0.343700


In [18]:
grouped = a.groupby(['시작지점의 위도', '시작지점의 경도'])
mean_values = grouped['열의 합'].mean()

In [21]:
b = mean_values.to_frame().reset_index()

In [23]:
b.to_csv('전체가중치.csv', encoding = 'utf-8-sig')